# Import and Cleaning

In [2]:
# Required Libraries
import pandas as pd
import sqlite3


In [4]:
# Load CSVs
menu_df = pd.read_csv('../data/fastfood.csv')
menu_df.head(3)

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380,60,7,2.0,0.0,95,1110,44,3.0,11,37.0,4.0,20.0,20.0,Other
1,Mcdonalds,Single Bacon Smokehouse Burger,840,410,45,17.0,1.5,130,1580,62,2.0,18,46.0,6.0,20.0,20.0,Other
2,Mcdonalds,Double Bacon Smokehouse Burger,1130,600,67,27.0,3.0,220,1920,63,3.0,18,70.0,10.0,20.0,50.0,Other


In [5]:
# Get information on df
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
 13  vit_a        301 non-null    float64
 14  vit_c        305 non-null    float64
 15  calcium      305 non-null    float64
 16  salad        515 non-null    object 
dtypes: float64(7), int64(7), object(3)
memory usage: 68.5+ KB


In [6]:
# Drop Columns missing several values
# Dropping Salad because it's not a useful field
menu_df = menu_df.drop(labels=['vit_a', 'vit_c', 'calcium', 'salad'], axis=1)

# Check to ensure it worked
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [7]:
# Clean Fiber and Protien missing values
# Calculate column averages
fiber_avg = menu_df['fiber'].mean()
protein_avg = menu_df['protein'].mean()

# Fill missing values with averages
menu_df['fiber'].fillna(fiber_avg, inplace=True)
menu_df['protein'].fillna(protein_avg, inplace=True)

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        515 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      515 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [8]:
# Convert all integer columns to floats to make it easier to compare
int_columns = menu_df.select_dtypes(include='int64').columns
menu_df[int_columns] = menu_df[int_columns].astype(float)

# Create a new column to display the item with the chain name after, making it easier to read the drop down menus
menu_df.insert(2, 'item_with_chain', menu_df['item'] + ' (' + menu_df['restaurant'] + ')')

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant       515 non-null    object 
 1   item             515 non-null    object 
 2   item_with_chain  515 non-null    object 
 3   calories         515 non-null    float64
 4   cal_fat          515 non-null    float64
 5   total_fat        515 non-null    float64
 6   sat_fat          515 non-null    float64
 7   trans_fat        515 non-null    float64
 8   cholesterol      515 non-null    float64
 9   sodium           515 non-null    float64
 10  total_carb       515 non-null    float64
 11  fiber            515 non-null    float64
 12  sugar            515 non-null    float64
 13  protein          515 non-null    float64
dtypes: float64(11), object(3)
memory usage: 56.5+ KB


In [9]:
# Find duplicate items in 'item_with_chain'
print(menu_df['item_with_chain'].value_counts())

duplicate_items = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]

# Display duplicate items
print(duplicate_items[['item_with_chain', 'calories', 'cal_fat', 'sat_fat', \
                       'trans_fat', 'cholesterol', 'sodium', 'total_carb', 'fiber', 'sugar', 'protein']])

# Remove one of the duplicates (you can choose 'first' or 'last')
menu_df = menu_df.drop_duplicates(subset='item_with_chain', keep='first')

Express Taco Salad w/ Chips (Taco Bell)         2
Chili Cheese Burrito (Taco Bell)                2
Artisan Grilled Chicken Sandwich (Mcdonalds)    1
6" Sweet Onion Chicken Teriyaki (Subway)        1
Footlong Subway Seafood Sensation (Subway)      1
                                               ..
Roast Turkey & Swiss Sandwich (Arbys)           1
Roast Beef Gyro (Arbys)                         1
Reuben Sandwich (Arbys)                         1
5 piece Prime-Cut Chicken Tenders (Arbys)       1
Fiesta Taco Salad-Steak (Taco Bell)             1
Name: item_with_chain, Length: 513, dtype: int64
                             item_with_chain  calories  cal_fat  sat_fat  \
414         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
492         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
497  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   
511  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   

     trans_fat  chol

In [10]:
# Check again
duplicate_items2 = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]
print(duplicate_items2)

Empty DataFrame
Columns: [restaurant, item, item_with_chain, calories, cal_fat, total_fat, sat_fat, trans_fat, cholesterol, sodium, total_carb, fiber, sugar, protein]
Index: []


In [11]:
# Look at shape
menu_df.shape

(513, 14)

In [12]:
# Look at columns
menu_df.columns

Index(['restaurant', 'item', 'item_with_chain', 'calories', 'cal_fat',
       'total_fat', 'sat_fat', 'trans_fat', 'cholesterol', 'sodium',
       'total_carb', 'fiber', 'sugar', 'protein'],
      dtype='object')

In [13]:
menu_df.insert(0, 'id', range(1, len(menu_df) + 1))


In [14]:
menu_df.head(2)

,id,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
0,1,Mcdonalds,Artisan Grilled Chicken Sandwich,Artisan Grilled Chicken Sandwich (Mcdonalds),380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0
1,2,Mcdonalds,Single Bacon Smokehouse Burger,Single Bacon Smokehouse Burger (Mcdonalds),840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0


### Save to clean CSV

In [16]:
menu_df.to_csv('../data/fastfood_cleaned.csv', index=False)


In [17]:
summary_table = menu_df.groupby('restaurant').mean()
summary_table

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_7747/3674030214.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summary_table = menu_df.groupby('restaurant').mean()


,id,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
restaurant,,,,,,,,,,,,
Arbys,165.0,532.727273,237.836364,26.981818,7.972727,0.418182,70.454545,1515.272727,44.872727,2.709091,7.563636,29.254545
Burger King,227.5,608.571429,333.757143,36.814286,11.150000,0.864286,100.857143,1223.571429,39.314286,2.633882,8.185714,29.984158
Chick Fil-A,71.0,384.444444,145.370370,16.148148,4.111111,0.037037,79.074074,1151.481481,28.629630,2.454606,4.148148,31.703704
Dairy Queen,283.5,520.238095,260.476190,28.857143,10.440476,0.678571,71.547619,1181.785714,38.690476,2.833333,6.357143,24.833333
Mcdonalds,29.0,640.350877,285.614035,31.807018,8.289474,0.464912,109.736842,1437.894737,48.789474,3.228070,11.070175,40.298246
Sonic,111.0,631.698113,338.301887,37.641509,11.415094,0.933962,86.981132,1350.754717,47.207547,2.660377,6.528302,29.188679
Subway,352.5,503.020833,165.104167,18.479167,6.197917,0.218750,61.302083,1272.968750,54.718750,6.562500,10.093750,30.312500
Taco Bell,457.0,443.008850,187.699115,20.858407,6.557522,0.243363,38.893805,1012.389381,46.575221,5.699115,3.690265,17.380531


In [18]:
summary_table = summary_table.drop('id', axis=1)
summary_table.head(2)

,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
restaurant,,,,,,,,,,,
Arbys,532.727273,237.836364,26.981818,7.972727,0.418182,70.454545,1515.272727,44.872727,2.709091,7.563636,29.254545
Burger King,608.571429,333.757143,36.814286,11.150000,0.864286,100.857143,1223.571429,39.314286,2.633882,8.185714,29.984158


In [19]:
summary_table.insert(0, 'id', range(1, len(summary_table) + 1))


In [20]:
summary_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, Arbys to Taco Bell
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8 non-null      int64  
 1   calories     8 non-null      float64
 2   cal_fat      8 non-null      float64
 3   total_fat    8 non-null      float64
 4   sat_fat      8 non-null      float64
 5   trans_fat    8 non-null      float64
 6   cholesterol  8 non-null      float64
 7   sodium       8 non-null      float64
 8   total_carb   8 non-null      float64
 9   fiber        8 non-null      float64
 10  sugar        8 non-null      float64
 11  protein      8 non-null      float64
dtypes: float64(11), int64(1)
memory usage: 832.0+ bytes


In [20]:
menu_df.to_csv('../data/summary_table.csv', index=False)
